# emotional tts quick c1 iteration

## Preparing EmotionalTTS dataset

In [ ]:
# downloading the datasets

# !(mkdir /DataEmotionalTTS && \

!(cd /DataEmotionalTTS && \
    wget https://www.openslr.org/resources/110/thorsten-emotional_v02.tgz && \
    tar -zxvf thorsten-emotional_v02.tgz)

In [ ]:
# create manifests, normalization and phonemization

# !(cd NeMoEmotionalTTS && \
#     python get_data.py \

!(python scripts/dataset_processing/tts/openslr_emotional/get_data.py \
        --data-root ../DataEmotionalTTS/ \
        --val-size 0.1 \
        --test-size 0.2)

In [ ]:
# 2cc2cc4a34b961ef1657cc82dbd18875 does not exist for whisper category

In [ ]:
# phonemization
## inside the phonemizer container

from phonemizer.backend import EspeakBackend
import json

backend = EspeakBackend('de')
unique_symbols = set()
input_manifest_filepaths = ["/DataEmotionalTTS/thorsten-emotional_v02/train_manifest", \
                            "/DataEmotionalTTS/thorsten-emotional_v02/test_manifest", \
                            "/DataEmotionalTTS/thorsten-emotional_v02/val_manifest"]

for input_manifest_filepath in input_manifest_filepaths:
    output_manifest_filepath = input_manifest_filepath+"_phonemes"
    records = []
    n_text = []
    with open(input_manifest_filepath + ".json", "r") as f:
        for i, line in enumerate(f):
            d = json.loads(line)
            records.append(d)
            n_text.append(d['normalized_text'])

    phonemized = backend.phonemize(n_text)
    for line in n_text:
        for char in line:
            unique_symbols.add(char)
    new_records = []
    for i in range(len(records)):
        records[i]["is_phoneme"] = 0
        new_records.append(records[i])
        phoneme_record = records[i].copy()
        phoneme_record["normalized_text"] = phonemized[i]
        phoneme_record["is_phoneme"] = 1
        new_records.append(phoneme_record)

    with open(output_manifest_filepath + ".json", "w") as f:
        for r in new_records:
            f.write(json.dumps(r) + '\n')
print(unique_symbols)

In [ ]:
a = ''.join(unique_symbols)

In [ ]:
a = a.lower()

In [ ]:
[char for char in a if char not in "ʊʃŋɜːɛɾəɪçɔøɡœɑÜ„1Q̃ɒʒÄɹÖʌθàó̈ðéɐáabcdefghijklmnopqrstuvwxyzäöüß"]

In [ ]:
!ls /DataEmotionalTTS/thorsten-emotional_v02

In [ ]:
# dataset config
!cat scripts/dataset_processing/tts/openslr_emotional/ds_conf/ds_for_fastpitch_align.yaml

In [ ]:
# supplementary data creation - emotional TTS (optional)
!(HYDRA_FULL_ERROR=1 python scripts/dataset_processing/tts/extract_sup_data.py \
        --config-path /NeMoEmotionalTTS/scripts/dataset_processing/tts/openslr_emotional/ds_conf \
        --config-name ds_for_fastpitch_align.yaml \
        manifest_filepath=/DataEmotionalTTS/thorsten-emotional_v02/train_manifest_phonemes.json \
        sup_data_path=/DataEmotionalTTS/thorsten-emotional_v02/phonemes/)

In [ ]:
# PITCH_MEAN=169.59390258789062, PITCH_STD=103.64842224121094
# PITCH_MIN=65.4063949584961, PITCH_MAX=2093.004638671875

## Preparing NeutralTTS dataset

In [ ]:
# edit the manifest files
## path corrections :%s/github\/datasets/DataEmotionalTTS/g
## add emotion_id =0 :%s/, "is_p/, "emotion_id": 0, "is_p/g

In [ ]:
# supplementary data creation - Neutral TTS (optional)
!(HYDRA_FULL_ERROR=1 PYTHONPATH=. python scripts/dataset_processing/tts/extract_sup_data.py \
        --config-path /NeMoEmotionalTTS/scripts/dataset_processing/tts/openslr/ds_conf \
        --config-name ds_for_fastpitch_align.yaml \
        manifest_filepath=/DataEmotionalTTS/openslr-95-german-neutral-tts/thorsten-de/train_manifest_phonemes.json \
        sup_data_path=/DataEmotionalTTS/openslr-95-german-neutral-tts/thorsten-de/phonemes6/)

In [ ]:
# PITCH_MEAN=132.55270385742188, PITCH_STD=37.61381912231445
# PITCH_MIN=65.4063949584961, PITCH_MAX=2093.004638671875

In [ ]:
# pre-train fastpitch (with speaker and emotional embedding) on German neutral tts
!(CUDA_VISIBLE_DEVICES=0 HYDRA_FULL_ERROR=1 PYTHONPATH=. python examples/tts/fastpitch.py --config-path conf/de --config-name fastpitch_align_22050 \
    name="c1e1-FastPitch-pretrain" \
    model.train_ds.dataloader_params.batch_size=32 \
    model.validation_ds.dataloader_params.batch_size=32 \
    train_dataset=/DataEmotionalTTS/openslr-95-german-neutral-tts/thorsten-de/train_manifest_phonemes.json \
    validation_datasets=/DataEmotionalTTS/openslr-95-german-neutral-tts/thorsten-de/val_manifest_phonemes.json \
    sup_data_path=/DataEmotionalTTS/openslr-95-german-neutral-tts/thorsten-de/phonemes6/ \
    whitelist_path=nemo_text_processing/text_normalization/de/data/whitelist.tsv \
    exp_manager.exp_dir=/resultGermanTTS \
    trainer.max_epochs=100 \
    pitch_mean=132.524658203125 \
    pitch_std=37.389366149902344 \
    +exp_manager.create_wandb_logger=true \
    +exp_manager.wandb_logger_kwargs.name="072720221625_c1e1-FastPitch-pretrain" \
    +exp_manager.wandb_logger_kwargs.project="EmotionalTTS" | tee -a 072720221625_c1e1-FastPitch-pretrain.log)

In [ ]:
# monitor performance of pre-trained fastpitch


In [ ]:
# finetune the model trained in the last cell
!(CUDA_VISIBLE_DEVICES=0 python examples/tts/fastpitch_finetune.py --config-path conf/de --config-name fastpitch_align_v1.05.yaml \
    name="c1e1-FastPitch-finetune" \
    model.train_ds.dataloader_params.batch_size=32 \
    model.validation_ds.dataloader_params.batch_size=32 \
    train_dataset=/DataEmotionalTTS/thorsten-emotional_v02/train_manifest_phonemes.json \
    validation_datasets=/DataEmotionalTTS/thorsten-emotional_v02/val_manifest_phonemes.json \
    sup_data_path=/DataEmotionalTTS/thorsten-emotional_v02/train_manifest_phonemes.json \
    whitelist_path=nemo_text_processing/text_normalization/de/data/whitelist.tsv \
    exp_manager.exp_dir=/resultEmotionalTTS \
    +init_from_nemo_model=/DataEmotionalTTS/tts_de_fastpitchhifigan_v1.10.0/tts_de_fastpitch_align.nemo \
    +trainer.max_steps=1000 ~trainer.max_epochs \
    trainer.check_val_every_n_epoch=25 \
    model.n_speakers=1 \
    model.n_emotions=8 \
    model.optim.lr=2e-4 \
    ~model.optim.sched model.optim.name=adamw trainer.devices=1 trainer.strategy=null \
    pitch_mean=169.59390258789062 \
    pitch_std=103.64842224121094 \
    +exp_manager.create_wandb_logger=true \
    +exp_manager.wandb_logger_kwargs.name="072620221311_c1e1-FastPitch-finetune" \
    +exp_manager.wandb_logger_kwargs.project="EmotionalTTS" | tee -a 072620221311_c1e1-FastPitch-pretrain.log)